In [2]:
%pip install git+https://github.com/denis-spe/exttorch.git

  Cloning https://github.com/denis-spe/exttorch.git to /tmp/pip-req-build-8yk0qnur
  Running command git clone --filter=blob:none --quiet https://github.com/denis-spe/exttorch.git /tmp/pip-req-build-8yk0qnur
  Resolved https://github.com/denis-spe/exttorch.git to commit 43afd2efff4742954c0ff3a395ee743d8a22f639
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for exttorch: filename=exttorch-0.0.1-py3-none-any.whl size=14209 sha256=f6c40db8dc0b88055de31b8ffb9f00ec206297d860c00932f5ad23383b93bded
  Stored in directory: /tmp/pip-ephem-wheel-cache-mtobhh0x/wheels/f5/02/a6/495ec4cdcef6d3da7eb7972076b23caf500318f58c2bb4afd8
Successfully built exttorch


**Iris**

In [14]:
import torch
from torch import nn
from torch.optim import Adam, SGD
from sklearn.datasets import load_iris
from exttorch.model import Sequential
from exttorch.metrics import Precision, Recall

In [4]:
x, y = load_iris(return_X_y=True)
print(x.shape, y.shape)

(150, 4) (150,)


In [7]:
set(y)

{0, 1, 2}

In [20]:
iris_model = Sequential([
    nn.Linear(4, 16),
    nn.ReLU(),
    nn.Linear(16, 16),
    nn.ReLU(),
    nn.Linear(16, 3)
])

iris_model.compile(
    optimizer=Adam(iris_model.parameters()),
    loss=nn.CrossEntropyLoss(),
    metrics=['accuracy',
            #  Precision(average="weighted"),
             Recall(average="weighted", zero_division=True)]
    )

iris_model.fit(x, y, epochs=10, batch_size=32, validation_data=(x, y))

Epoch 1/10
150/150 [==============================] - 1s 8ms/step - loss: 1.1266 - accuracy: 0.2800 - Recall: 0.2800 - val_loss: 1.1496 - val_accuracy: 0.3333 - val_Recall: 0.3333
Epoch 2/10
150/150 [==============================] - 1s 6ms/step - loss: 1.0136 - accuracy: 0.3533 - Recall: 0.3533 - val_loss: 1.0184 - val_accuracy: 0.3333 - val_Recall: 0.3333
Epoch 3/10
150/150 [==============================] - 1s 6ms/step - loss: 0.8869 - accuracy: 0.5600 - Recall: 0.5600 - val_loss: 0.8612 - val_accuracy: 0.3333 - val_Recall: 0.3333
Epoch 4/10
150/150 [==============================] - 1s 6ms/step - loss: 0.6894 - accuracy: 0.7733 - Recall: 0.7733 - val_loss: 1.0617 - val_accuracy: 0.5933 - val_Recall: 0.5933
Epoch 5/10
150/150 [==============================] - 1s 6ms/step - loss: 0.4711 - accuracy: 0.8800 - Recall: 0.8800 - val_loss: 1.0556 - val_accuracy: 0.5467 - val_Recall: 0.5467
Epoch 6/10
150/150 [==============================] - 1s 6ms/step - loss: 0.3720 - accuracy: 0.9267 